# Text generation using Amazon Titan - a foundation model for Generative AI

This sample implemetation is going to use a generative AI pre-trained model to create text summarisation. 

In [1]:
# Let's first configure AWS environment to connect with the foundation model 
# NOT RECOMMENDED FOR PRODUCTION DEPLOYMENT OR FOR SHARING IN PUBLIC.
import os
# os.environ['AWS_ACCESS_KEY_ID'] = 'Your key id'
# os.environ['AWS_SECRET_ACCESS_KEY'] = 'Your key'

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [2]:
# import boto3 to access AWS resources using python
import boto3

In [3]:
# bedrock client to explore the available foundation models
bedrock = boto3.client(service_name='bedrock',region_name='us-east-1')

In [4]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': 'c7cd2acd-ca8b-47e2-8bbb-b96fcd57aedc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 07 May 2025 04:39:44 GMT',
   'content-type': 'application/json',
   'content-length': '54644',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c7cd2acd-ca8b-47e2-8bbb-b96fcd57aedc'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0',
   'modelId': 'amazon.titan-image-generator-v1:0',
   'modelName': 'Titan Image Generator G1',
   'providerName': 'Amazon',

### Let's use `amazon.titan-text-express-v1` model for this example.

In [5]:
# To get information about this model
bedrock.get_foundation_model(modelIdentifier='amazon.titan-text-express-v1')

{'ResponseMetadata': {'RequestId': 'c6d0b89a-6902-4db8-8900-3be6e3c900dc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 07 May 2025 04:39:45 GMT',
   'content-type': 'application/json',
   'content-length': '524',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c6d0b89a-6902-4db8-8900-3be6e3c900dc'},
  'RetryAttempts': 0},
 'modelDetails': {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-text-express-v1',
  'modelId': 'amazon.titan-text-express-v1',
  'modelName': 'Titan Text G1 - Express',
  'providerName': 'Amazon',
  'inputModalities': ['TEXT'],
  'outputModalities': ['TEXT'],
  'responseStreamingSupported': True,
  'customizationsSupported': [],
  'inferenceTypesSupported': ['ON_DEMAND'],
  'modelLifecycle': {'status': 'ACTIVE'}}}

### Step 1: Prepare input for the model
Input format and model parameters can be obtained from the documentaion. Example - https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-text.html 
```
{
    "inputText": string,
    "textGenerationConfig": {
        "temperature": float,  
        "topP": float,
        "maxTokenCount": int,
        "stopSequences": [string]
    }
}
```

In [6]:
# Lets use json module to construct the input
import json

In [7]:
prompt = 'What is the highest mountain in the world?'
req = json.dumps({
    'inputText': prompt})
req

'{"inputText": "What is the highest mountain in the world?"}'

### Step 2: Create bedrock runtime

In [8]:
bedrock_runtime = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

### Step 3: Invoke the model and get response

In [9]:
response = bedrock_runtime.invoke_model(body=req, 
                                        modelId='amazon.titan-text-express-v1',
                                        contentType='application/json', 
                                        accept = 'application/json')

In [10]:
response

{'ResponseMetadata': {'RequestId': '0b9e77c6-bc88-4d21-a201-c197f0281ede',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 07 May 2025 04:39:49 GMT',
   'content-type': 'application/json',
   'content-length': '224',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0b9e77c6-bc88-4d21-a201-c197f0281ede',
   'x-amzn-bedrock-invocation-latency': '3731',
   'x-amzn-bedrock-output-token-count': '41',
   'x-amzn-bedrock-input-token-count': '9'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'body': <botocore.response.StreamingBody at 0x106350d60>}

In [11]:
response_body = json.loads(response.get("body").read())
response_body

{'inputTextTokenCount': 9,
 'results': [{'tokenCount': 41,
   'outputText': '\nMount Everest, the highest mountain on Earth, is located in the Himalayas. The mountain is 8,848 meters (29,029 feet) tall.',
   'completionReason': 'FINISH'}]}

In [12]:
print(response_body['results'][0]['outputText'])


Mount Everest, the highest mountain on Earth, is located in the Himalayas. The mountain is 8,848 meters (29,029 feet) tall.
